Connect with the APIs for Anthropic and Google, as well as OpenAI.

ANTHROPIC_API_KEY=xxxx

In [4]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
import json
from typing import List
import google.generativeai
from IPython.display import Markdown, display, update_display

In [5]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
google_api_key = os.getenv('GOOGLE_API_KEY')
    
if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:7]}")
else:
    print("Google API Key not set")

Google API Key exists and begins AIzaSyB


In [6]:
# This is the set up code for Gemini
# Having problems with Google Gemini setup? Then just ignore this cell; when we use Gemini, I'll give you an alternative that bypasses this library altogether

google.generativeai.configure()

In [7]:
system_message = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists"

In [8]:
# The API for Gemini has a slightly different structure.
# I've heard that on some PCs, this Gemini code causes the Kernel to crash.
# If that happens to you, please skip this cell and use the next cell instead - an alternative approach.

gemini = google.generativeai.GenerativeModel(
    model_name='gemini-2.0-flash-exp',
    system_instruction=system_message
)
response = gemini.generate_content(user_prompt)
print(response.text)

Why did the data scientist break up with the time series analyst?

Because she said he was too predictable and only ever talked about future trends! He just couldn't see their relationship having a positive autocorrelation.



In [15]:
gemini = google.generativeai.GenerativeModel(
    model_name='gemini-2.0-flash-exp',
    system_instruction=system_message
)
response = gemini.generate_content(user_prompt)

generated_text = ""
while True:
 response = gemini.generate_content(prompt + generated_text, # Add previously generated text
                                            max_output_tokens=100)  # Limit the output
 chunk = response.text
 if not chunk: # End of generation
     break
 generated_text += chunk
 yield chunk # Yield the chunk
 if len(chunk) < chunk_size: # Check if the model has stopped
    break
print (generated_text) # Return the full text at the end

SyntaxError: 'yield' outside function (4125706138.py, line 15)

In [18]:
##Create a Project to scrap NTT Data web site and provide the company Brocuhure

In [43]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [44]:
ed = Website("https://www.nttdata.com/global/en/")
ed.links

['/global/en',
 '/global/en/industries',
 '/global/en/industries/automotive',
 '/global/en/industries/energy-utilities',
 '/global/en/industries/financial-services',
 '/global/en/industries/healthcare',
 '/global/en/industries/higher-education-research',
 '/global/en/industries/insurance',
 '/global/en/industries/life-sciences-pharma',
 '/global/en/industries/manufacturing',
 '/global/en/industries/public-sector',
 '/global/en/industries/retail-cpg',
 '/global/en/industries/telecom-media-technology',
 '/global/en/industries/travel-transportation-logistics',
 '/global/en/services',
 '/global/en/services/application-services',
 '/global/en/services/business-process-services',
 '/global/en/services/cloud',
 '/global/en/services/business-consulting',
 '/global/en/services/cx-digital-products',
 '/global/en/services/cybersecurity',
 '/global/en/services/data-and-intelligence',
 '/global/en/services/digital-workplace',
 '/global/en/services/edge',
 '/global/en/services/global-data-centers',


In [83]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""
link_system_prompt += "Return the response in JSON format with complete object"

In [84]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [85]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [96]:
def get_links(url):
    website = Website(url)
    response = claude.messages.create(
    model="claude-3-5-sonnet-latest",
    max_tokens=600,
    temperature=0.7,
    system=link_system_prompt,
    messages=[
        {"role": "user", "content": get_links_user_prompt(website)},
    ],
)
    result = response.content[0].text
    return json.loads(result)

In [104]:
##ntt.links

In [97]:
print(get_links("https://www.nttdata.com/global/en/"))

{'links': [{'type': 'about page', 'url': 'https://www.nttdata.com/global/en/about-us'}, {'type': 'company profile', 'url': 'https://www.nttdata.com/global/en/about-us/profile'}, {'type': 'company history', 'url': 'https://www.nttdata.com/global/en/about-us/ntt-data-history'}, {'type': 'leadership', 'url': 'https://www.nttdata.com/global/en/about-us/leaders'}, {'type': 'careers page', 'url': 'https://www.nttdata.com/global/en/careers'}, {'type': 'sustainability', 'url': 'https://www.nttdata.com/global/en/about-us/sustainability'}, {'type': 'news', 'url': 'https://www.nttdata.com/global/en/news'}, {'type': 'contact', 'url': 'https://www.nttdata.com/global/en/contact-us'}, {'type': 'investors', 'url': 'https://www.nttdata.com/global/en/investors'}]}


In [154]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    ##print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [103]:
##print(get_all_details("https://www.nttdata.com/global/en/"))

In [113]:
system_brochure_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."



In [101]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [114]:
get_brochure_user_prompt("NTTDATA", "https://www.nttdata.com/global/en/")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.nttdata.com/global/en/about-us'}, {'type': 'company profile', 'url': 'https://www.nttdata.com/global/en/about-us/profile'}, {'type': 'company history', 'url': 'https://www.nttdata.com/global/en/about-us/ntt-data-history'}, {'type': 'leadership', 'url': 'https://www.nttdata.com/global/en/about-us/leaders'}, {'type': 'careers page', 'url': 'https://www.nttdata.com/global/en/careers'}, {'type': 'sustainability', 'url': 'https://www.nttdata.com/global/en/about-us/sustainability'}, {'type': 'news', 'url': 'https://www.nttdata.com/global/en/news'}, {'type': 'contact', 'url': 'https://www.nttdata.com/global/en/contact-us'}, {'type': 'investor relations', 'url': 'https://www.nttdata.com/global/en/investors'}]}


"You are looking at a company called: NTTDATA\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHome | NTT DATA Group\nWebpage Contents:\nIndustries\nIndustries\nAutomotive\nEnergy & Utilities\nFinancial Services\nHealthcare\nHigher Education & Research\nInsurance\nLife Sciences & Pharma\nManufacturing\nPublic Sector\nRetail & CPG\nTelecom, Media & Technology\nTravel, Transportation & Logistics\nServices\nServices\nApplication Services\nBusiness Process Services\nCloud\nConsulting\nCX and Digital Products\nCybersecurity\nData & Artificial Intelligence\nDigital Workplace\nEdge\nGlobal Data Centers\nNetwork Services\nSustainability Services\nTech Solutions & Integration\nInsights\nBy type\nBlog: NTT DATA Focus\nClients Cases\nEvents\nBy topic\nThe World Economic Forum, Davos 2025\nTechnology\nGenerative AI\nADM Technology Focus Areas\nUnlock the Full Potential of 

In [115]:
def create_brochure(company_name, url):
    response = claude.messages.create(
    model="claude-3-5-sonnet-latest",
    max_tokens=600,
    temperature=0.7,
    system=system_brochure_prompt,
    messages=[
        {"role": "user", "content": get_brochure_user_prompt(company_name, url)},
    ],
)
    result = response.content[0].text
    display(Markdown(result))

In [116]:
create_brochure("NTTDATA", "https://www.nttdata.com/global/en/")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.nttdata.com/global/en/about-us'}, {'type': 'company profile', 'url': 'https://www.nttdata.com/global/en/about-us/profile'}, {'type': 'company history', 'url': 'https://www.nttdata.com/global/en/about-us/ntt-data-history'}, {'type': 'leadership', 'url': 'https://www.nttdata.com/global/en/about-us/leaders'}, {'type': 'sustainability', 'url': 'https://www.nttdata.com/global/en/about-us/sustainability'}, {'type': 'careers page', 'url': 'https://www.nttdata.com/global/en/careers'}, {'type': 'news', 'url': 'https://www.nttdata.com/global/en/news'}, {'type': 'contact page', 'url': 'https://www.nttdata.com/global/en/contact-us'}]}


# NTT DATA Group - Corporate Brochure

## Who We Are
NTT DATA is a global technology and business solutions provider that specializes in digital transformation and IT services. As part of the NTT Group, we combine deep industry expertise with cutting-edge technology to deliver innovative solutions to clients worldwide.

## Our Services & Solutions
### Core Offerings:
- Application Services
- Cloud Solutions
- Consulting Services
- Cybersecurity
- Data & Artificial Intelligence
- Digital Workplace Solutions
- Sustainability Services

### Industry Focus:
We serve multiple sectors including:
- Financial Services
- Healthcare
- Manufacturing
- Public Sector
- Retail
- Automotive
- Insurance
- Telecom & Technology

## Innovation & Technology
NTT DATA is at the forefront of technological innovation, with particular expertise in:
- Generative AI
- Cloud Computing
- Digital Transformation
- Sustainable Technology Solutions

## Global Presence
With operations across multiple continents, NTT DATA provides localized expertise with global reach, serving clients in various markets and industries worldwide.

## Corporate Values
Our approach is guided by:
- Client-First Mindset
- Innovation Focus
- Sustainability Commitment
- Digital Transformation Excellence

## Strategic Partnerships
We maintain strong partnerships with industry leaders including:
- Microsoft
- SAP
- Salesforce
- AWS
- Cisco

## Corporate Citizenship
We are committed to:
- Sustainability Initiatives
- Diversity, Equity & Inclusion
- Social Responsibility
- Environmental Stewardship

## Career Opportunities
NTT DATA offers:
- Global Career Paths
- Professional Development
- Innovation-Focused Environment
- Diverse and Inclusive Workplace

## Notable Recognitions
- Leader in Global Generative AI Services
- Strong presence at major technology events including MWC Barcelona
- Sponsor of major sporting events including The Open and NTT INDYCAR SERIES

---
*For more information about career opportunities, partnerships, or services, visit our website or contact our regional offices.*

In [174]:
def stream_brochure(company_name, url):
    result = claude.messages.create(
    model="claude-3-5-sonnet-latest",
    max_tokens=1000,
    temperature=0.7,
    stream=True,  # Enable streaming
    system=system_brochure_prompt,
    messages=[
        {"role": "user", "content": get_brochure_user_prompt(company_name, url)},
    ],
)
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in result:
        if chunk.type == 'content_block_delta':
            response += chunk.delta.text or ''
            response = response.replace("```","").replace("markdown", "")
            update_display(Markdown(response), display_id=display_handle.display_id)

       

In [175]:
stream_brochure("NTTDATA", "https://www.nttdata.com/global/en/")

# NTT DATA - Transforming Business Through Technology
![NTT DATA Logo]

## Who We Are
NTT DATA is a global technology and business solutions provider that helps organizations accelerate their digital journeys. As part of the NTT Group, we combine deep industry expertise with cutting-edge technology to deliver practical and scalable IT solutions that transform businesses and society.

## Our Services & Solutions
### Core Offerings:
- Application Services
- Cloud Solutions
- Consulting Services
- Cybersecurity
- Data & Artificial Intelligence
- Digital Workplace Solutions
- Network Services
- Sustainability Services

### Industry Expertise:
We serve multiple sectors including:
- Financial Services
- Healthcare
- Manufacturing
- Public Sector
- Retail
- Automotive
- Insurance
- Telecom & Technology

## Innovation Focus
NTT DATA is at the forefront of technological innovation, with particular emphasis on:
- Generative AI
- Cloud Computing
- Digital Transformation
- Sustainability Solutions
- Advanced Data Analytics

## Global Presence
With operations worldwide, NTT DATA delivers consistent, high-quality services while maintaining local market understanding and client relationships.

## Partnerships
We maintain strategic partnerships with industry leaders including:
- Microsoft
- SAP
- Salesforce
- AWS
- Cisco

## Corporate Values
Our mission is driven by core principles that emphasize:
- Innovation
- Client Success
- Sustainable Solutions
- Global Collaboration
- Social Responsibility

## Notable Engagements
NTT DATA maintains high-profile sponsorships including:
- The Open Championship
- AIG Women's Open
- NTT INDYCAR SERIES

## Career Opportunities
We offer exciting career paths for professionals interested in:
- Technology Consulting
- Software Development
- Project Management
- Digital Innovation
- Business Analysis

---
*For more information about careers, partnerships, or services, visit www.nttdata.com*